In [4]:
import sys
import bs4 as bs
import urllib.request
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import lxml
import seaborn as sns
import sktime
from kats.consts import TimeSeriesData

### Collecting the flow data of the Cheat River from the Albright Gauge # 03070260

### Things I want to investigate:
* How many times a year does the Albright gauge read over 20,000 cfs
* Can I predict Albright gauge levels from upstream gauges
    * Can I create a machine learning model that will account for change in level and transmit that downstream

In [ ]:
source = urllib.request.urlopen('https://nwis.waterservices.usgs.gov/nwis/iv/?format=waterml,2.0&sites=03070260&startDT'
    '=2018-01-01T00:00-0500&endDT=2021-12-31T23:59-0500&parameterCd=00060&siteType=ST&siteStatus=all').read()
soup = bs.BeautifulSoup(source, 'lxml')

# There appears to be a bottleneck so I had to do this in batches to achieve a sample from 2010 - 2021

In [ ]:
# Setting up the variable to catch the data
cfs = []
timestamp = []

In [ ]:
# gatering the cfs and timestamp data
def acquire_data():
    for wml2 in soup.find_all('wml2:value'):
      cfs.append(float(wml2.string))
    for wml2 in soup.find_all('wml2:time'):
      timestamp.append(wml2.string)

    return cfs, timestamp

In [ ]:
# Collecting the data
cfs, timestamp = acquire_data()

In [ ]:
cfs[:10], timestamp[:10]

In [ ]:
data_dict = {"timestamp":timestamp, "CFS":cfs}


In [ ]:
df_2018 = pd.DataFrame(data_dict)
df_2018.head()

In [ ]:
df_2018.tail()

In [ ]:
data_2014 = {"timestamp":timestamp, "CFS":cfs}
df_2014 = pd.DataFrame(data_2014)
df_2014.head()

In [ ]:
data_2010 = {"timestamp":timestamp, "CFS":cfs}
df_2010 = pd.DataFrame(data_2010)
df_2010.head()

In [ ]:
big_df = pd.concat([df_2010, df_2014, df_2018])
big_df.head(), big_df.tail()

### I will save the data, and use that from here on out - since collecting is a bit of a tedious chore 

In [ ]:
big_df.to_csv("2010-2022_alley_cfs.csv", index=False)

In [ ]:
big_df.shape

In [ ]:
big_df.info()

In [ ]:
big_df.isna().sum()

In [ ]:
big_df.describe()

In [ ]:
big_df["CFS"].sum()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(big_df["timestamp"][:1000], big_df["CFS"][:1000])

In [ ]:
big_df["CFS"].plot.hist()

In [ ]:
big_df.timestamp.dtype

In [ ]:
big_df.head()

## Parse out timestamp

I will do this by importing the csv data and parse dates vis timestamp column

In [ ]:
df = pd.read_csv("2010-2022_alley_cfs.csv",
                low_memory=False)

In [ ]:
df.timestamp.dtype

In [ ]:
df.head()

In [ ]:
df.sort_values(by=["timestamp"], inplace=True, ascending=True)
df.timestamp.head()

In [ ]:
df_tmp = df.copy()

### I realize now, that the parse dates import did not work

I will try another method

In [ ]:
df_tmp["timestamp"] = pd.to_datetime(df_tmp["timestamp"].str.slice(stop=19), format='%Y-%m-%dT%H:%M:%S', utc=True)

In [ ]:
df_tmp.info()

In [ ]:
df_tmp.head()

In [ ]:
df_tmp["Year"] = df_tmp["timestamp"].dt.year
df_tmp["Month"] = df_tmp["timestamp"].dt.month
df_tmp["Day"] = df_tmp["timestamp"].dt.day
df_tmp["DayofWeek"] = df_tmp["timestamp"].dt.dayofweek
df_tmp["DayofYear"] = df_tmp["timestamp"].dt.dayofyear

In [ ]:
df_tmp.head()

In [ ]:
df_tmp["Hour"] = df_tmp["timestamp"].dt.hour

In [ ]:
df_tmp.head()

In [ ]:
df_tmp["Minute"] = df_tmp["timestamp"].dt.minute
df_tmp.head()

In [ ]:
(df_tmp["CFS"]>20000).value_counts()

### I want to find how many days a year the cfs is over 20k for every year

In [ ]:
df_tmp["Flood"] = df_tmp["CFS"]>20000
df_tmp.head()

In [ ]:
(df_tmp["Flood"]==True).value_counts()

So a total number of entries of flooding = 2237

In [ ]:
occur = df_tmp.groupby(["Flood", "Year", "DayofYear"]).size()
occur

In [ ]:
pd.crosstab(df_tmp.Flood, df_tmp.Year)

In [ ]:
pd.crosstab(df_tmp.Year, df_tmp.Flood).plot(kind="bar",
                                    figsize=(20, 6),
                                    color=["plum", "teal"])
plt.title("Number of Readings above 20000 in a Year")
plt.xlabel("Year")
plt.ylabel("Number of Flood Readings")
plt.legend(["No Flood", "Flood"])
plt.xticks(rotation=0)
plt.grid(linestyle='-', axis='y');

In [ ]:
df_tmp["Flood"] = df_tmp["Flood"].astype(int)
df_tmp.head()

In [ ]:
plt.figure(figsize=(10, 6))

# Scatter with positive examples
plt.scatter(df_tmp.Year[df_tmp.Flood==1],
            df_tmp.Month[df_tmp.Flood==1],
            color="teal")

# Add some helpful info
plt.title("Year and Month of Flood")
plt.xlabel("Year")
plt.ylabel("Month");

In [ ]:
df_tmp.groupby(["Month", "Flood"]).count()

In [ ]:
df_tmp.describe()

In [ ]:
g = sns.lmplot(x="Year", y="Month", hue="Flood", data=df_tmp)

In [ ]:
df_tmp["Alley"] = (df_tmp["CFS"]>500) & (df_tmp["CFS"]<2000)
df_tmp.head()

In [ ]:
df_tmp["Alley"].value_counts()

In [ ]:
# Make a DataFrame of single day values that are the mean values for that day
single_day_value = df_tmp.groupby(pd.Grouper(key="timestamp", freq="1d")).mean()
single_day_value.head(5), df_tmp.head(5)

In [ ]:
single_day_value[:5]

In [ ]:
df_tmp[:20]

In [ ]:
single_day_value["Flood"].value_counts()

In [ ]:
single_day_value["Flood"] = (single_day_value["Flood"]>0)
single_day_value["Flood"].value_counts()

In [ ]:
single_day_value["Flood"].astype(int)
single_day_value["Flood"].value_counts()

In [ ]:
pd.crosstab(single_day_value.Year, single_day_value.Flood).plot(kind="bar",
                                             xlabel="Year",
                                             ylabel="Number of Days with CFS value over 20000")

In [ ]:
flooded = single_day_value[single_day_value["Flood"]==True]
flooded.head()

In [ ]:
flooded.groupby(pd.Grouper(key="Year")).sum()

In [ ]:
df_tmp.groupby(pd.Grouper(key="Year")).sum()["Flood"]

In [ ]:
single_day_value.groupby(pd.Grouper(key="Year")).sum()

In [ ]:
for_graph = flooded.groupby(pd.Grouper(key="Year")).count()
for_graph.head()

In [ ]:
for_graph.reset_index(inplace=True)
for_graph.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(for_graph.Year, for_graph.Flood)
plt.xlabel("Year")
plt.ylabel("Number of Days With Mean CFS over 20,000")
plt.title("Analysis of Days Experiencing Flood Events at Albright USGS Gauge Over Time")
plt.xticks(np.arange(2010, 2022, step=1))
plt.yticks(np.arange(0, 16, step=1));

In [ ]:
# I am trying to figure out how to make the above graph from the original DataFrame
#fig, ax = plt.subplots(figsize=(10, 6))
#ax.bar(single_day_value["Year"], single_day_value.groupby(["Year"], squeeze=True).count()["Flood"])
#plt.xlabel("Year")
#plt.ylabel("Number of Days With Mean CFS over 20,000")
#plt.title("Analysis of Days Experiencing Flood Events at Albright USGS Gauge Over Time")
#plt.xticks(np.arange(2010, 2022, step=1))
#plt.yticks(np.arange(0, 16, step=1));

### I am curious about 2018!

Let's investigate that year

In [ ]:
albright_2018 = single_day_value[single_day_value["Year"]==2018]
albright_2018

In [ ]:
albright_2018.plot(y="CFS")

In [ ]:
albright_2018["CFS"].mean()

In [ ]:
albright_2018.describe()

In [ ]:
sept_2018 = albright_2018[albright_2018["Month"]==9.0]


In [ ]:
sept_2018

In [ ]:
sept_2018.plot(x="Day",
               y="CFS",
              xticks=np.arange(0, 31),
              figsize=(10, 6))

## I want to collect sample data from multiple sites and see what I have to do to fit the different gauge sites together onto the same DataFrame indexed by a single datetime column

Setup the beautiful soup data extraction workflow

In [1]:
# gatering the cfs and timestamp data
def acquire_data():
    for wml2 in soup.find_all('wml2:value'):
      cfs.append(float(wml2.string))
    for wml2 in soup.find_all('wml2:time'):
      timestamp.append(wml2.string)

    return cfs, timestamp

In [5]:
# Create a dictionary of sites and their id numbers
site_list = {"Albright":'03070260',
             "Parsons": '03069500',
             "DryFork": '03065000'
             }

In [6]:
# Create a loop to grab data for multiple sites placing the data in a variable
# called the site name
for i in site_list:
    source = urllib.request.urlopen(f'https://nwis.waterservices.usgs.gov/nwis/iv/?format=waterml,2.0&sites={site_list[i]}&startDT'
    '=2018-01-01T00:00-0500&endDT=2021-12-31T23:59-0500&parameterCd=00060&siteType=ST&siteStatus=all').read()
    soup = bs.BeautifulSoup(source, 'lxml')
    cfs = []
    timestamp = []
    # Collecting the data
    cfs, timestamp = acquire_data()
    # Creating variable name to hold the data
    exec('{KEY} = {VALUE}'.format(KEY = i, VALUE = dict(zip(timestamp, cfs))))
    

In [7]:
Albright

{'2018-01-01T05:45:00-05:00': 1070.0,
 '2018-01-01T07:45:00-05:00': 1060.0,
 '2018-01-01T09:45:00-05:00': 1040.0,
 '2018-01-01T11:45:00-05:00': 1030.0,
 '2018-01-01T13:45:00-05:00': 1020.0,
 '2018-01-01T15:45:00-05:00': 1010.0,
 '2018-01-01T17:45:00-05:00': 995.0,
 '2018-01-01T19:45:00-05:00': 985.0,
 '2018-01-01T21:45:00-05:00': 974.0,
 '2018-01-01T23:45:00-05:00': 963.0,
 '2018-01-02T01:45:00-05:00': 953.0,
 '2018-01-02T03:45:00-05:00': 943.0,
 '2018-01-02T05:45:00-05:00': 934.0,
 '2018-01-02T07:45:00-05:00': 925.0,
 '2018-01-02T09:45:00-05:00': 915.0,
 '2018-01-02T11:45:00-05:00': 906.0,
 '2018-01-02T13:45:00-05:00': 897.0,
 '2018-01-02T15:45:00-05:00': 887.0,
 '2018-01-02T17:45:00-05:00': 878.0,
 '2018-01-02T19:45:00-05:00': 880.0,
 '2018-01-02T21:45:00-05:00': 882.0,
 '2018-01-02T23:45:00-05:00': 885.0,
 '2018-01-03T01:45:00-05:00': 888.0,
 '2018-01-03T03:45:00-05:00': 890.0,
 '2018-01-03T05:45:00-05:00': 893.0,
 '2018-01-03T07:45:00-05:00': 893.0,
 '2018-01-03T09:45:00-05:00': 89

In [8]:
Parsons

{'2018-01-01T05:00:00-05:00': 641.0,
 '2018-01-01T05:15:00-05:00': 641.0,
 '2018-01-01T05:30:00-05:00': 640.0,
 '2018-01-01T05:45:00-05:00': 639.0,
 '2018-01-01T06:00:00-05:00': 638.0,
 '2018-01-01T06:15:00-05:00': 637.0,
 '2018-01-01T06:30:00-05:00': 636.0,
 '2018-01-01T06:45:00-05:00': 635.0,
 '2018-01-01T07:00:00-05:00': 634.0,
 '2018-01-01T07:15:00-05:00': 633.0,
 '2018-01-01T07:30:00-05:00': 632.0,
 '2018-01-01T07:45:00-05:00': 631.0,
 '2018-01-01T08:00:00-05:00': 630.0,
 '2018-01-01T08:15:00-05:00': 629.0,
 '2018-01-01T08:30:00-05:00': 628.0,
 '2018-01-01T08:45:00-05:00': 628.0,
 '2018-01-01T09:00:00-05:00': 627.0,
 '2018-01-01T09:15:00-05:00': 626.0,
 '2018-01-01T09:30:00-05:00': 625.0,
 '2018-01-01T09:45:00-05:00': 624.0,
 '2018-01-01T10:00:00-05:00': 623.0,
 '2018-01-01T10:15:00-05:00': 622.0,
 '2018-01-01T10:30:00-05:00': 621.0,
 '2018-01-01T10:45:00-05:00': 620.0,
 '2018-01-01T11:00:00-05:00': 619.0,
 '2018-01-01T11:15:00-05:00': 618.0,
 '2018-01-01T11:30:00-05:00': 617.0,
 

In [9]:
DryFork

{'2018-01-01T05:45:00-05:00': 294.0,
 '2018-01-01T06:45:00-05:00': 292.0,
 '2018-01-01T07:45:00-05:00': 290.0,
 '2018-01-01T08:45:00-05:00': 288.0,
 '2018-01-01T09:45:00-05:00': 287.0,
 '2018-01-01T10:45:00-05:00': 284.0,
 '2018-01-01T11:45:00-05:00': 282.0,
 '2018-01-01T12:45:00-05:00': 280.0,
 '2018-01-01T13:45:00-05:00': 278.0,
 '2018-01-01T14:45:00-05:00': 277.0,
 '2018-01-01T15:45:00-05:00': 276.0,
 '2018-01-01T16:45:00-05:00': 276.0,
 '2018-01-01T17:45:00-05:00': 275.0,
 '2018-01-01T18:45:00-05:00': 274.0,
 '2018-01-01T19:45:00-05:00': 273.0,
 '2018-01-01T20:45:00-05:00': 272.0,
 '2018-01-01T21:45:00-05:00': 272.0,
 '2018-01-01T22:45:00-05:00': 271.0,
 '2018-01-01T23:45:00-05:00': 270.0,
 '2018-01-02T00:45:00-05:00': 269.0,
 '2018-01-02T01:45:00-05:00': 268.0,
 '2018-01-02T02:45:00-05:00': 268.0,
 '2018-01-02T03:45:00-05:00': 267.0,
 '2018-01-02T04:45:00-05:00': 266.0,
 '2018-01-02T05:45:00-05:00': 265.0,
 '2018-01-02T06:45:00-05:00': 264.0,
 '2018-01-02T07:45:00-05:00': 263.0,
 

The different sites don't have similar time schedules

In [ ]:
len(Albright), len(Parsons), len(DryFork)

In [ ]:
df = pd.DataFrame.from_dict(Albright, orient='index')

In [ ]:
df

In [ ]:
df["cfs"] = df[0]
df.head()

In [ ]:
df.drop(0, axis=1)

In [ ]:
df["Albright"] = df["cfs"]
df.drop("cfs", axis=1, inplace=True)
df.drop(0, axis=1, inplace=True)
df.head()

In [ ]:
df["timestamp"] = df.index
df.head()

In [ ]:
df["Parsons"] = df["timestamp"].map(Parsons)

In [ ]:
df.head()

In [ ]:
Parsons

In [ ]:
df["DryFork"] = df["timestamp"].map(DryFork)
df.head()

In [ ]:
len(df)

In [ ]:
df["timestamp"] = pd.to_datetime(df['timestamp'])
df.head()

In [ ]:
type(df.index)

In [ ]:
df.index = pd.to_datetime(df.index, utc=True)

In [ ]:
type(df.index)

In [ ]:
df.head()

In [ ]:
df.drop("timestamp", axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
Albright["2018-01-01T05:45:00-05:00"], Parsons["2018-01-01T05:45:00-05:00"], DryFork["2018-01-01T05:45:00-05:00"]

I am going to save the dataframe so I can come back and use this
I think it is setup how I want it

In [ ]:
df.to_csv("albright-parsons-dryfork2018-2021.csv")

In [ ]:
df.tail()

In [ ]:
# Test the csv
df1 = pd.read_csv('albright-parsons-dryfork2018-2021.csv', index_col=0)
df1.head()

In [ ]:
df1.index.name = "Date"

In [ ]:
df1.head()

In [ ]:
df1.plot(figsize=(10, 6))

In [ ]:
# I want to label the index
df1.to_csv("albright-parsons-dryfork2018-2021.csv")

In [ ]:
df1.head()

In [ ]:
df1.index

In [ ]:
df1.index = pd.to_datetime(df1.index, utc=True)

In [ ]:
df1.index = df1.index.tz_localize(None)
df1.head()

In [ ]:
df1.index

In [ ]:
# NOW I want to save the csv.... everything is how I want
df1.to_csv("albright-parsons-dryfork2018-2021.csv")

### OKAY, ALmost done setting up the data I think.

Things to do:
    * Look at data - I bet there are NaN
    * Set up data into train test etc.

In [ ]:
df = pd.read_csv('albright-parsons-dryfork2018-2021.csv', index_col="Date")

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df["Dates"] = df.index

In [ ]:
fig, ax = plt.subplots()
ax.plot(df["Dates"][:1000], df["Albright"][:1000])

In [ ]:
df.isnull().sum()/len(df)

In [ ]:
len(df)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
len(df)

### Get sktime set up

In [ ]:
from sktime.utils.plotting import plot_series
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.compose import ColumnEnsembleForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.datatypes import check_raise
from sktime.datatypes import convert
from sktime.datatypes import mtype
from sktime.datatypes import check_is_mtype

In [ ]:
df.drop("Dates", axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.isna().sum()

In [ ]:
type(df)

In [ ]:
type(df.index)

In [ ]:
df.index = pd.PeriodIndex(df.index, freq="T")

In [ ]:
type(df.index)

In [ ]:
df.head()

In [ ]:
example = get_examples(mtype="pd.DataFrame", as_scitype="Series")[0]

In [ ]:
example

In [ ]:
type(example)

In [ ]:
type(example.index)

In [ ]:
df = df.reset_index()

In [ ]:
type(df.index)

In [ ]:
df.head()

In [ ]:
df.drop("Date", axis=1, inplace=True)

In [ ]:
df

In [ ]:
type(df.Albright)

In [ ]:
y = df

In [ ]:
forecaster.fit(y)

In [ ]:
y.drop(["Parsons", "DryFork"], axis=1, inplace=True)

In [ ]:
y.head()

In [ ]:
plot_series(y)

In [ ]:
forecaster.fit(y)

In [ ]:
fh = np.arange(1, 10000)
fh

In [ ]:
y_pred = forecaster.predict(fh)

In [ ]:
plot_series(y[136400:], y_pred, labels=["y", "y_pred"])

In [ ]:
# step 3: specifying the forecasting algorithm
forecaster = NaiveForecaster(strategy="last", sp=12)

# step 4: fitting the forecaster
forecaster.fit(y)

# step 5: querying predictions
y_pred = forecaster.predict(fh)

In [ ]:


# optional: plotting predictions and past data
plot_series(y[136000:], y_pred, labels=["y", "y_pred"])



In [ ]:
y

In [ ]:
y["Parsons"] = df["Parsons"]

In [ ]:
y

In [ ]:
from sktime import DecisionTreeRegressor

In [ ]:
forecasters = [
    ("trend", PolynomialTrendForecaster(), 0),
    ("ses", ExponentialSmoothing(trend="add"), 1),
]

forecaster = ColumnEnsembleForecaster(forecasters=forecasters)
forecaster.fit(y, fh=[1, 2, 3])

y_pred = forecaster.predict()

In [ ]:
type(y)

In [ ]:


from sktime.datatypes import check_is_mtype


In [ ]:
type(a.index)

In [ ]:
type(y.index)

In [ ]:
type(a.a)

In [ ]:
type(y.Albright)

In [ ]:
mtype(y)

## Let's try using Kats... sktime is tough for me

In [ ]:
df.head()

In [ ]:
df.to_csv("for_kats.csv")

In [ ]:
df = pd.read_csv("for_kats.csv")

In [ ]:
df_ts = TimeSeriesData(time=df.Date, value=df[['Albright', 'Parsons', 'DryFork']], date_format="%Y%m%d %H%M%S")

In [ ]:
type(df_ts)

In [ ]:
df.Date

In [ ]:
df_ts[1:5] + df_ts[1:5]

In [ ]:
%matplotlib inline

df_ts.plot()

In [ ]:
df_ts.plot(cols=['Albright', 'Parsons'])

In [ ]:
type(df_ts)
df_ts